<a href="https://colab.research.google.com/github/jaya-shankar/DSS_project/blob/main/DSS_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
!rm -rf DSS_project

!git clone https://github.com/jaya-shankar/DSS_project.git

Cloning into 'DSS_project'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 66 (delta 37), reused 47 (delta 20), pack-reused 0
Unpacking objects: 100% (66/66), done.


In [22]:
import pandas as pd

In [23]:
root_path = "/content/DSS_project/raw_datasets"

In [24]:
df = pd.read_csv(f"{root_path}/COVID-19_Hospital_Data_Coverage_Detail.csv")
df

,state,ccn,facility_name,street_address,city,zip_code,fips_county,provider_subtype,reporting_source,dates_with_missing_data,...,on_hand_supply_therapeutic_b_bamlanivimab_courses,previous_week_therapeutic_b_bamlanivimab_courses_used,previous_week_personnel_covid_vaccinated_doses_administered,total_personnel_covid_vaccinated_doses_none,total_personnel_covid_vaccinated_doses_one,total_personnel_covid_vaccinated_doses_all,total_personnel,previous_week_patients_covid_vaccinated_doses_one,previous_week_patients_covid_vaccinated_doses_all,Georeference
0,AZ,030108,"CORE INSTITUTE SPECIALTY HOSPITAL, THE",6501 NORTH 19TH AVENUE,PHOENIX,85015.0,13,Short Term,tt,[],...,True,True,True,True,True,True,True,True,True,POINT (-112.099612 33.531909)
1,CO,061310,SEDGWICK COUNTY MEMORIAL HOSPITAL,900 CEDAR ST,JULESBURG,80737.0,115,Critical Access Hospitals,tt,[],...,True,True,True,True,True,True,True,True,True,POINT (-102.269232 40.993743)
2,FL,102012,PAM HEALTH SPECIALTY HOSPITAL OF JACKSONVILLE,4901 RICHARD ST,JACKSONVILLE,32207.0,31,Long Term,tt,[],...,True,True,True,True,True,True,True,True,True,POINT (-81.616762 30.271029)
3,IA,160124,LAKES REGIONAL HEALTHCARE,2301 HIGHWAY 71,SPIRIT LAKE,51360.0,59,Short Term,tt,[],...,False,False,True,True,True,True,True,True,True,POINT (-95.126311 43.418176)
4,IN,151320,IU HEALTH JAY,500 W VOTAW ST,PORTLAND,47371.0,75,Critical Access Hospitals,tt,[],...,True,True,True,True,True,True,True,True,True,POINT (-84.983969 40.439368)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5851,LA,191310,FRANKLIN FOUNDATION HOSPITAL,1097 NORTHWEST BLVD,FRANKLIN,70538.0,101,Critical Access Hospitals,state,[],...,False,False,True,True,True,True,True,True,True,POINT (-91.524776 29.802231)
5852,PA,390201,LEHIGH VALLEY HOSPITAL - POCONO,206 EAST BROWN STREET,EAST STROUDSBURG,18301.0,89,Short Term,tt,[],...,False,False,False,True,True,True,True,False,False,POINT (-75.174327 40.992284)
5853,WI,520038,AURORA MEDICAL CTR WASHINGTON COUNTY,1032 E SUMNER ST,HARTFORD,53027.0,131,Short Term,tt,[],...,False,False,False,False,False,False,False,False,False,POINT (-88.362376 43.317603)
5854,GA,110010,EMORY UNIVERSITY HOSPITAL,"1364 CLIFTON ROAD, NE",ATLANTA,30322.0,89,Short Term,tt,[],...,True,True,True,True,True,True,True,True,True,POINT (-84.320977 33.791881)


In [17]:
df.provider_subtype.unique()

array(['Short Term', 'Critical Access Hospitals', 'Long Term',
       'Psychiatric', 'Rehabilitation', 'Childrens Hospitals',
       "Medicaid Only Children's Psychiatric"], dtype=object)

In [73]:
general_df = pd.read_csv(f"{root_path}/hospital_count.csv")

In [77]:
general_df_states = set(general_df.state_code)
general_df_states
general_df.set_index("state_code",inplace=True)

## Adding roads data

In [ ]:
roads_df = pd.read_csv(f"{root_path}/roads_measure.csv",encoding='utf-16' )
roads_df

In [ ]:
roads_dic = {}
for state in general_df_states:
  try:
    roads_dic[state] = float(roads_df[roads_df.code==state].Value)
  except:
    roads_dic[state] = None
roads_dic

In [86]:
general_df["miles_of_road"] = 0
for i in general_df.index:
  general_df["miles_of_road"][i] = roads_dic[i]

## Adding politics data

In [90]:
politics_df = pd.read_csv(f"{root_path}/democratic_vs_republican_votes_by_usa_state_2020.csv")
politics_df.columns

Index(['state', 'DEM', 'REP', 'usa_state', 'usa_state_code',
       'percent_democrat'],
      dtype='object')

In [ ]:
dem_dic = {}
for state in general_df_states:
  try:
    dem_dic[state] = float(politics_df[politics_df.usa_state_code==state].percent_democrat)
  except:
    dem_dic[state] = None
dem_dic

In [ ]:
general_df["percent_democrat"] = 0
for i in general_df.index:
  general_df["percent_democrat"][i] = dem_dic[i]

## Adding Population Data

In [132]:
population_df = pd.read_csv(f"{root_path}/PopulationEstimates.csv")
population_df.columns

Index(['FIPStxt', 'State', 'Area name', 'Rural-urban continuum code 2013',
       'Population 2020'],
      dtype='object')

In [148]:
population_df.dropna(subset = ['Population 2020'], inplace=True)

In [149]:
def sum_population(state_code):
  # print(state_code)
  
  l = list(population_df[population_df.State==state_code]['Population 2020'])
  f = lambda x : int(x.replace(",",""))
  l = list(map(f,l))
  p = sum(l)
  return p

In [ ]:
popu_dic = {}
for state in general_df_states:

  popu_dic[state] = sum_population(state)
popu_dic

In [151]:
general_df["population"] = 0
for i in general_df.index:
  general_df["population"][i] = popu_dic[i]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [154]:
general_df.sort_index(level=1)

,no_of_hospitals,miles_of_road,percent_democrat,population
state_code,,,,
AK,16,17680.0,36.098993,1466782
AL,114,100170.0,37.032892,10048558
AR,106,99246.0,35.609218,6023048
AZ,107,66968.0,50.000000,14303004
CA,397,175562.0,65.934760,79076446
CO,105,89207.0,56.000000,11547428
CT,39,21575.0,60.241900,7211888
DC,12,1515.0,94.707520,1379090
DE,15,6525.0,59.646859,1979896
